## This code is to find out the different of each epoch and batch size combination

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Path

In [2]:
'''find the Model path'''
# find the current path
current_path = os.getcwd()
print('The current path is:', current_path)

# find the parent path
parent_path = Path(current_path).parent
print('The parent path is:', parent_path)

# find the data path
data_path = parent_path / 'Data/alexnet/fashion_mnist/'
print('The data path is:', data_path)

The current path is: /home/yj/FinalThesis/GreenAI/Cloud/3060_95W/code
The parent path is: /home/yj/FinalThesis/GreenAI/Cloud/3060_95W
The data path is: /home/yj/FinalThesis/GreenAI/Cloud/3060_95W/Data/alexnet/fashion_mnist


### extract data

In [3]:
batch_sizes = [64, 128, 256, 512]
epochs = [10, 20, 30, 40, 50]
rounds = 5

#### read all the data from all the folders

In [4]:
folder_list = []
for epoch in epochs:
    for batch in batch_sizes:
        for round in range(rounds):
            folder_name = 'E' + str(epoch) + '_B' + str(batch) + '_R' + str(round)
            # print('The folder name is:', folder_name)
            folder_list.append(folder_name)
            
# find the folders in the data path
folder_names = os.listdir(data_path)
print('The folder names are:', folder_names)
print('The folder list is:', folder_list)

# find the common folders
common_folders = list(set(folder_list).intersection(folder_names))
# sort the common folders
common_folders.sort()
print('The common folders are:', common_folders)
print(len(common_folders))

The folder names are: ['E30_B64_R4', 'E30_B256_R3', 'E20_B64_R4', 'E20_B128_R1', 'E30_B512_R2', 'E20_B256_R4', 'E10_B64_R1', 'E30_B512_R1', 'E10_B512_R3', 'E10_B64_R3', 'E30_B256_R2', 'E10_B256_R3', 'E10_B128_R2', 'E10_B256_R4', 'E20_B64_R1', 'E30_B512_R4', 'E20_B512_R0', 'E30_B64_R2', 'E30_B64_R0', 'E20_B256_R2', 'E10_B256_R1', 'E10_B64_R2', 'E10_B512_R4', 'E20_B64_R3', 'E10_B512_R0', 'E30_B128_R4', 'E20_B128_R3', 'E10_B128_R4', 'E20_B64_R0', 'E20_B512_R3', 'E30_B512_R3', 'E30_B128_R0', 'E20_B256_R1', 'E20_B128_R2', 'E30_B256_R4', 'E10_B256_R0', 'E20_B256_R0', 'E10_B64_R4', 'E20_B512_R4', 'E10_B512_R2', 'E20_B256_R3', 'E30_B512_R0', 'E40_B64_R0', 'E30_B256_R0', 'E30_B128_R3', 'E20_B512_R1', 'E20_B512_R2', 'E20_B128_R0', 'E30_B128_R1', 'E10_B128_R3', 'E20_B64_R2', 'E10_B128_R0', 'E10_B64_R0', 'E30_B64_R3', 'E10_B128_R1', 'E20_B128_R4', 'E30_B128_R2', 'E30_B64_R1', 'E30_B256_R1', 'E10_B256_R2', 'E10_B512_R1']
The folder list is: ['E10_B64_R0', 'E10_B64_R1', 'E10_B64_R2', 'E10_B64_R3', '

### analysis data

#### find the data in same epoch and batch size

In [7]:
name_dic = {
    'EpochEnergy': 'Epoch_energy',
    'EpochTime': 'Epoch_time',
    'TestAcc': 'Test_acc',
    'TrainAcc': 'Train_acc',
    'LayerTime': 'Layer_time',
    'TrainTime': 'Train_time',
    'TrainPartTime': 'Train_part_time'
}

In [17]:
# find the folders that have same epoch and batch size in common_folders
same_epoch_batch = []
for folder in common_folders:
    epoch = int(folder.split('_')[0][1:])
    batch = int(folder.split('_')[1][1:])
    same_epoch_batch.append((epoch, batch))
print('The same epoch and batch size are:', same_epoch_batch)

# find in each combination of epoch and batch size, find out the folder names
epoch_batch_folders = {}
for epoch, batch in same_epoch_batch:
    epoch_batch_folders[(epoch, batch)] = []
    for folder in common_folders:
        if int(folder.split('_')[0][1:]) == epoch and int(folder.split('_')[1][1:]) == batch:
            epoch_batch_folders[(epoch, batch)].append(folder)
print('The epoch and batch size folders are:', epoch_batch_folders)
# for each combination of epoch and batch size, find the average test accuracy, and the average energy consumption
for eb_comb in epoch_batch_folders:
    print('The epoch and batch size combination is:', eb_comb)
    energy_acc = {}
    for folder in epoch_batch_folders[eb_comb]:
        files_path = data_path / folder
        files = os.listdir(files_path)
        # print('The files in the folder are:', files_path, files)
        # test_acc = []
        for f in files:
            name = f.split('.')[0]
            file_path = files_path / f
            if name in name_dic['TestAcc']:
                TestAcc = np.load(file_path, allow_pickle=True)
                # print('The test accuracy is:', TestAcc)
            elif name in name_dic['EpochEnergy']:
                EpochEnergy = np.load(file_path, allow_pickle=True)
                # print('The energy consumption is:', EpochEnergy)
            elif name in name_dic['EpochTime']:
                EpochTime = np.load(file_path, allow_pickle=True)
                # print('The epoch time is:', EpochTime)
            for energy in EpochEnergy:
                # find out the each epoch energy consumption
                
                
                
                
                
                
                
                
                
                
                
    #             test_acc.append(TestAcc)
    # epoch_batch_folders[eb_comb] = np.mean(test_acc)

The same epoch and batch size are: [(10, 128), (10, 128), (10, 128), (10, 128), (10, 128), (10, 256), (10, 256), (10, 256), (10, 256), (10, 256), (10, 512), (10, 512), (10, 512), (10, 512), (10, 512), (10, 64), (10, 64), (10, 64), (10, 64), (10, 64), (20, 128), (20, 128), (20, 128), (20, 128), (20, 128), (20, 256), (20, 256), (20, 256), (20, 256), (20, 256), (20, 512), (20, 512), (20, 512), (20, 512), (20, 512), (20, 64), (20, 64), (20, 64), (20, 64), (20, 64), (30, 128), (30, 128), (30, 128), (30, 128), (30, 128), (30, 256), (30, 256), (30, 256), (30, 256), (30, 256), (30, 512), (30, 512), (30, 512), (30, 512), (30, 512), (30, 64), (30, 64), (30, 64), (30, 64), (30, 64), (40, 64)]
The epoch and batch size folders are: {(10, 128): ['E10_B128_R0', 'E10_B128_R1', 'E10_B128_R2', 'E10_B128_R3', 'E10_B128_R4'], (10, 256): ['E10_B256_R0', 'E10_B256_R1', 'E10_B256_R2', 'E10_B256_R3', 'E10_B256_R4'], (10, 512): ['E10_B512_R0', 'E10_B512_R1', 'E10_B512_R2', 'E10_B512_R3', 'E10_B512_R4'], (10, 6